In [1]:
import sys

from loguru import logger

from anki_ai.domain.model import Deck

In [2]:
logger.remove()
logger.add(sys.stderr, level="ERROR")

1

When exporting notes from Anki, select the following options:
- Notes in plain text (.txt)
- Include HTML and media references
- Include tags
- Include deck name
- Include notetype name
- Include unique identifier

This is important to later be able to overwrite the notes in our deck, with the changes we made, without losing all of the statistics. 

In [3]:
deck = Deck()
deck.read_txt("../data/Selected Notes.txt")
print(f"The deck contains {len(deck)} notes")

The deck contains 2241 notes


One issue we need to handle is that our notes might contain HTML and media references, which can confuse the LLM when editing the notes. In our deck, these are quite frequent, because we are using the [Markdown and KaTeX Support (Rework) - Code Highlighting, Math, ... add-on](https://ankiweb.net/shared/info/1786114227). Here is an example:

In [4]:
deck[0].back

'```bash<br>$ ln -s &lt;file_name&gt; &lt;link_name&gt;<br>```'

In [5]:
from io import StringIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def replace_br_with_newline(html_string):
    import re
    return re.sub(r'<br\s*/?>', '\n', html_string)

def strip_tags(html):
    s = MLStripper()
    s.feed(replace_br_with_newline(html))
    return s.get_data()

In [6]:
back_card = deck[0].back
print(f"Original note:\n{back_card}\n")
print(f"Fixed:\n{strip_tags(back_card)}")

Original note:
```bash<br>$ ln -s &lt;file_name&gt; &lt;link_name&gt;<br>```

Fixed:
```bash
$ ln -s <file_name> <link_name>
```


### vLLM

Please run the following command on the terminal to spin up a vLLM server, which we will query in the rest of the notebook.


```bash
vllm serve meta-llama/Meta-Llama-3.1-8B-Instruct --max_model_len 4096 --chat-template ./template_llama31.jinja
```

We might have to wait about 1 minute before the server is up and reachable.

In [7]:
from vllm import LLM, SamplingParams
from openai import OpenAI

In [8]:
# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

In [9]:
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [10]:
def vllm_generate(prompt: str, n_notes: int=10):
    for i in range(n_notes):
        user_msg = f"""Front: {deck[i].front}
        Back: {deck[i].back}
        """
    
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ]

        chat_response = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct",
            messages=messages,
            temperature=0,
        )
    
        print("#######################")
        print(f"Front: {deck[i].front}\nBack: {deck[i].back}")
        print(chat_response.choices[0].message.content)

In [11]:
system_msg = """
Optimize this Anki note:
- Concise, simple, distinct
- Follow format rules

Reply in this format:
Front: [edited front]
Back: [edited back]

Terminal commands:
```bash
$ command <placeholder>
```

Code:
```language
code here
```

Use the following placeholders only: <file>, <path>, <link>, <command>.

No explanations.

Example 1:
Front: What command does extract files from a zip archive?
Back: ```bash
$ unzip <file>
```
Front: Extract zip files
Back: ```bash
$ unzip <file>
```

Example 2:
Front: What is the command to print manual or get help for a command?
Back: ```bash
$ man ...
```
Front: Get command manual/help
Back: ```bash
$ man <command>
```

Example 3: 
Front: What command does create a soft link?
Back: ```bash
$ ln -s <file_name> <link_name>
```
Front: Create soft link
Back: ```bash
$ ln -s <file> <link>
```

Example 4:
Front: In the `ln -s` command, what is the order of file name and link name?
Back: ```bash
$ ln -s <file_name> <link_name>
```
Front: `ln -s` argument order
Back: <file> then <link>
"""

vllm_generate(system_msg, n_notes=25)

#######################
Front: What command does create a soft link?
Back: ```bash<br>$ ln -s &lt;file_name&gt; &lt;link_name&gt;<br>```
Front: Create soft link
Back: ```bash
$ ln -s <file> <link>
```
#######################
Front: In the `ln -s` command, what is the order of file name and link name?
Back: ```bash<br>$ ln -s &lt;file_name&gt; &lt;link_name&gt;<br>```
Front: `ln -s` argument order
Back: <file> then <link>
#######################
Front: What command does extract files from a zip archive?
Back: ```bash<br>$ unzip &lt;file&gt;<br>```
Front: Extract zip files
Back: ```bash
$ unzip <file>
```
#######################
Front: What is the command to list the content of a directory?
Back: ```bash<br>$ ls &lt;path&gt;<br>```
Front: List directory content
Back: ```bash
$ ls <path>
```
#######################
Front: What is the command to print text to the terminal window?
Back: ```bash<br>$ echo ...<br>```
Front: Print text to terminal
Back: ```bash
$ echo ...
```
#################